In [ ]:
import sys
sys.path.append("../..")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Jupyter stuff
# You may need to conda install seaborn matplotlib tqdm jupyterlab
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm

In [ ]:
import random
import numpy as np

from tangle.core import Tangle, Transaction
from tangle.core.tip_selection import MaxDepthTipSelector
from tangle.theory import TempTransactionStore, TheoreticalNode


In [ ]:
NUM_NODES = 1000
NUM_ROUNDS = 150
NODES_PER_ROUND = 10

DIST_STD_DEV = 100

PARTICLES_DEPTH = 50

In [ ]:
# Data
mu, sigma = 0, DIST_STD_DEV # mean and standard deviation
node_data = [np.array(x) for x in np.random.normal(mu, sigma, NUM_NODES)]

In [ ]:
# Tangle Initialization
tx_store = TempTransactionStore()
genesis = Transaction([])
# Initialize the genesis transaction with a number within the std dev interval
tx_store.save(genesis, np.array(random.uniform(-DIST_STD_DEV, DIST_STD_DEV)))
tangle = Tangle({genesis.id: genesis}, genesis.id)

In [ ]:
# Results
num_txs = []     # Number of published transactions per round
consensus = []   # Value of consensus transaction per round (should converge to 0)

In [ ]:
for r in tqdm(range(NUM_ROUNDS)):
    txs = []

    for _ in range(NODES_PER_ROUND):
        node_id = np.random.randint(NUM_NODES)
        tip_selector = MaxDepthTipSelector(tangle, PARTICLES_DEPTH)
        node = TheoreticalNode(tangle, tx_store, tip_selector, node_id, None, node_data[node_id])
        tx, tx_weights = node.create_transaction()

        if tx is not None:
            tx_store.save(tx, tx_weights)
            txs.append(tx)

    for tx in txs:
        tangle.add_transaction(tx)  

    tip_selector = MaxDepthTipSelector(tangle, PARTICLES_DEPTH)
    # Use 'the average' node (data=0) for validation
    validation_node = TheoreticalNode(tangle, tx_store, tip_selector, 0, None, 0)
    reference_txs, reference = validation_node.obtain_reference_params()

    num_txs.append(len(txs))
    consensus.append(np.linalg.norm(reference))

In [ ]:
sns.lineplot(data=pd.DataFrame(num_txs))

In [ ]:
plt.plot(consensus)

In [ ]:
consensus[-1], min(consensus)